# Neste lab vamos criar um classificador utilizando NLTK e Redes Neurais

Vamos utilizar o mesmo dataset de revisão de filmes disponível em http://www.cs.cornell.edu/people/pabo/movie-review-data/

Segue o processo de criação do classificador:

1) Vamos treinar modelos com avaliações positivas e negativas. Para isso vamos selecionar 70% dos dados para treinamento 
2) Depois que o modelo estiver treinado, vamos utilizar os 30% restantes para testar o modelo, passando apenas o texto e verificando se a previsão bateu com a categoria alvo (positiva ou negativa)

Para esta prática baixamos no exercício anterior o dataset http://www.cs.cornell.edu/people/pabo/movie-review-data/mix20_rand700_tokens_cleaned.zip

Baixe e descompacte este arquivo zip. Se já fez isso no exercício anterior pode pular esse passo.

In [ ]:
# Instalando módulos novos
! pip install numpy
! pip install sklearn
! pip install tensorflow
! pip install tflearn

In [ ]:
# Importações Iniciais
import os
import nltk
import time
import random
import numpy as np
import tensorflow as tf
import tflearn
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split 

In [ ]:
# Definição de variaveis globais
FILES_DIR = 'tokens'
MINIMO_LETRAS = 3
PALAVRAS_IGNORADAS = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']','``',':','http','html','//members']
NUMERO_PALAVRAS_MAIS_FREQUENTES = 1000
PERCENTUAL_TESTE = 0.33
PASSOS_TREINAMENTO = 100
BATCH_SIZE = 50
VALIDATION_SET = 0.1

# Leitura dos textos e categorias

In [ ]:
start = time.time()
categorias = []
documentos = []
resultados = []
i = 0
for _, dirs, _ in os.walk(os.path.join(FILES_DIR)):
    for d in dirs:
        if d not in categorias:
            categorias.append(d)
        print('lendo categoria {}'.format(d))
        for _, _, files in os.walk(os.path.join(FILES_DIR,d)):
            for f in files:
                i += 1
                if i % 100 == 0:
                    print('{} arquivos lidos. lendo {}'.format(i,f))
                with open(os.path.join(FILES_DIR,d,f), "r") as f:
                     documentos.append(f.read())
                resultados.append(d)
end = time.time()                
print ('\nExistem {} textos e {} categorias: {}. \nProcessamento em {}s'.format(len(documentos),len(categorias),categorias,end - start))                                

## Preparação do texto

O que vamos fazer em sequida é preparar o texto de cada arquivo, fazendo a tokenização, stopwords e lemmatization

Além disso vamos calcular as 1000 palavras mais frequentes em todos os textos. Para isso precisamos de armazenar em uma variável todas as palavras que apareceram em todos os documentos

## Tokenização

In [ ]:
# Tokenização de palavras
start = time.time()
documentos_tokenizados = []
for documento in documentos:
    documentos_tokenizados.append(word_tokenize(documento))    
end = time.time()
print ('{} documentos tokenizados. \nProcessamento em {}s'.format(len(documentos_tokenizados),end - start))                                

In [ ]:
# Imprimindo o primeiro documento após processamento
print(documentos_tokenizados[0])

## Stop Words

In [ ]:
# Removendo Stop Words, lista de ignore e palavras menores que N letras
# Stop words em inglês
start = time.time()
english_stops = set(stopwords.words('english'))
documentos_sem_stop_words_e_ignore = []
for documento in documentos_tokenizados:
    # List comprehension para remover as stop words
    aux = [w for w in documento if w not in english_stops]
    # List comprehension para remover as demais palavras a serem ignoradas
    aux2 = [w for w in aux if w not in PALAVRAS_IGNORADAS]
    # List comprehension para remover palavras menores que N letras
    documentos_sem_stop_words_e_ignore.append([w for w in aux2 if len(w) >= MINIMO_LETRAS])
end = time.time()
print ('{} documentos removidos as stop words e palavras ignoradas. \nProcessamento em {}s'
       .format(len(documentos_sem_stop_words_e_ignore),end - start))

In [ ]:
# Imprimindo o primeiro documento após processamento
print(documentos_sem_stop_words_e_ignore[0])

## Stemming

In [ ]:
# Processando o stemming para cada palavra, transformando em minísculo
start = time.time()
stemmer = SnowballStemmer('english')
documentos_apos_stemming = []
for documento in documentos_sem_stop_words_e_ignore:
    # List comprehension para fazer o stemming de cada palavra
    documentos_apos_stemming.append([stemmer.stem(w.lower()) for w in documento])
end = time.time()
print ('{} documentos após aplicar stemmer. \nProcessamento em {}s'.format(len(documentos_apos_stemming),end - start))   

In [ ]:
# Imprimindo o primeiro documento após processamento
print(documentos_apos_stemming[0])

## Frequência

In [ ]:
# Primeiro vamos ter que criar uma lista de todas as palavras
start = time.time()
todas_palavras = []
for documento in documentos_apos_stemming:
    todas_palavras.extend(documento)
end = time.time()
print ('foram reunidas um todal de {} palavras. \nProcessamento em {}s'.format(len(todas_palavras),end - start))

In [ ]:
# Calculando a frequência
start = time.time()
fdist = nltk.FreqDist(todas_palavras)
palavras_mais_frequentes = []
for word_freq in fdist.most_common(NUMERO_PALAVRAS_MAIS_FREQUENTES):
    palavras_mais_frequentes.append(word_freq[0])
palavras_mais_frequentes = sorted(list(set(palavras_mais_frequentes)))
end = time.time()
print('Número total de tokens distintos {}. \nSelecionando apenas as {} palavras mais frequentes. \nProcessamento em {}s'
      .format(fdist.N(),NUMERO_PALAVRAS_MAIS_FREQUENTES,end - start))

In [ ]:
# Imprimindo apenas as 100 primeiras mais frequentes
print(palavras_mais_frequentes[:100])

# Bag of Words

A partir de agora já temos:

- palavras_mais_frequentes - lista de palavras mais frequentes
- textos_apos_stemming - Lista de documentos após pré-processamento
- resultados - Lista de resultados esperados
- categorias - Lista das possíveis categorias

Com isso, podemos agora criar um "saco de palavras" (bag of words) considerando apenas as N palavras mais frequentes

In [ ]:
# Vamos criar o BagOfWords (BoW) para os documentos, 
# considerando apenas as N palavras mais frequentes e ignorando as demais
start = time.time()
documentos_bow = []
resultados_bow = []
for i in range(len(documentos_apos_stemming)):
    documento = documentos_apos_stemming[i]
    resultado = resultados[i]
    documento_bow = []
    # Cria um array com o bag of words de N palavras mais frequentes, colocando 1 quando a palavra estiver presente no documento
    for w in palavras_mais_frequentes:
        documento_bow.append(1) if w in documento else documento_bow.append(0)
    documentos_bow.append(documento_bow)
    # Inicializa o array de resultados com zero
    resultado_bow = list([0] * len(categorias))
    # Coloca 1 na posição do array correta de acordo com o resultado
    resultado_bow[categorias.index(resultado)] = 1
    resultados_bow.append(resultado_bow)
end = time.time()
print('Criou um total de {} documentos BoW. \nProcessamento em {}s'.format(len(documentos_bow),end - start))

In [ ]:
# Imprimindo o primeiro documento BoW
print('primeiro documento {}'.format(documentos_bow[0]))

In [ ]:
# Imprimindo o primeiro documento BoW
print('primeiro resultado BoW {} para as categorias {}'.format(resultados_bow[0], categorias))

## Convertendo para array Numpy

In [ ]:
# Em seguida vamos converter para o formato nparray aceito pelo TensorFlow
documentos_bow = np.array(documentos_bow)
resultados_bow = np.array(resultados_bow)

## Separando dados de treinamento e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(documentos_bow, resultados_bow, test_size=PERCENTUAL_TESTE, random_state=42)
print('Separando {} documentos/resultados para treinamento e {} para teste'.format(len(X_train),len(X_test)))

## Criando o modelo da Rede Neural

In [ ]:
## Criando a Rede Neural utilizando a biblioteca TFLearn
# Reset do grafo
tf.reset_default_graph()
# Cria a rede neural
net = tflearn.input_data(shape=[None, len(X_train[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
net = tflearn.regression(net)
# Define o modelo e configura o tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs/'+time.strftime("%Y%m%d_%H%M"))
print('Modelo de rede neural criado: {}'.format(model))

## Realizando o Treinamento

In [ ]:
# Treinamento
model.fit(X_train, y_train, n_epoch=PASSOS_TREINAMENTO, batch_size=BATCH_SIZE, validation_set=VALIDATION_SET, show_metric=True)

## Tensorboard

Abra o **TensorBoard** utilizando o comando abaixo em um prompt do Python para acompanhar o treinamento.

Veja os gráficos de custo e acurácia gerados:
    
    python -m tensorboard.main --logdir="tflearn_logs"

## Avaliando o modelo treinado com os dados de teste

In [ ]:
score = model.evaluate(X_test, y_test)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))

In [ ]:
## Fazendo a inferência para um caso apenas
prediction = model.predict([X_test[0]])
print("Valor previsto: {}. \nValor esperado: {}".format(prediction[0],y_test[0]))

# Exercício 2

## Melhore o percentual do classificador 
    
Como pode ver no exemplo acima, o resultado do treinamento ainda não está com um percentual bom.

Realize as mudanças abaixo para tentar alcançar um perncentual melhor para o resultado da avaliação com os dados de teste

1) Faça alterações nos parâmetros abaixo e anote o resultado final de acurácia em uma tabela

    MINIMO_LETRAS
    PALAVRAS_IGNORADAS
    NUMERO_PALAVRAS_MAIS_FREQUENTES
    PERCENTUAL_TESTE
    PASSOS_TREINAMENTO
    BATCH_SIZE
    VALIDATION_SET

 2) Mude o desenho da rede neural e anote o resultado final de acurácia em uma tabela  

Salve o Jupyter Notebook com os resultados e entregue pelo iLang

# Tabela de resultados finais

Exemplo de tabela de resultado a ser gerada com valores fictícios:

| **Parâmetros alterados** | **Accuracy** |
| --- | --- | 
| VALORES INICIAIS | 69% |
| PASSOS_TREINAMENTO = 200 | 70% |
| PASSOS_TREINAMENTO = 200, BATCH_SIZE = 20 | 71% |



